# Configuration

In [1]:
%load_ext autoreload
%autoreload 2

## Read packages

In [2]:
import json, math, random, time, zipfile, io
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, Tuple, List, Optional
import copy

import numpy as np
import pandas as pd
from PIL import Image, ImageDraw

import torch, torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler
from torchvision import transforms, models

from torchmetrics.classification import BinaryAccuracy, BinaryAUROC

import albumentations as A
from albumentations.pytorch import ToTensorV2


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

import os
import sys

sys.path.append("..")

Device: cpu


## Read data-images

In [3]:
filename = "cnn_personalized.ipynb"  # Name of current file
print(f"Current absolute path: {os.getcwd()}\n")

# We specify the path of the current directory and the data and output directories.
ACTUAL_DIR = os.path.dirname(os.path.abspath(filename))
BASE_DIR = os.path.dirname(ACTUAL_DIR)
DATA_DIR = os.path.join(BASE_DIR, "data")
IMG_DIR = os.path.join(DATA_DIR, "images")
OUTPUT_DIR = os.path.join(DATA_DIR, "output")

print(f"BASE_DIR: {BASE_DIR}")
print(f"DATA_DIR: {DATA_DIR}")
print(f"IMG_DIR: {IMG_DIR}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")

Current absolute path: /Users/jhonattan.reales/Library/Mobile Documents/com~apple~CloudDocs/MAESTRIA MIAA 2025-2026/Cursos semestres/SEMESTRE 2/Aprendizaje automatico IV/Competencia-CNN/Kaggle_CNN_competition_ML4_MIAA/notebooks

BASE_DIR: /Users/jhonattan.reales/Library/Mobile Documents/com~apple~CloudDocs/MAESTRIA MIAA 2025-2026/Cursos semestres/SEMESTRE 2/Aprendizaje automatico IV/Competencia-CNN/Kaggle_CNN_competition_ML4_MIAA
DATA_DIR: /Users/jhonattan.reales/Library/Mobile Documents/com~apple~CloudDocs/MAESTRIA MIAA 2025-2026/Cursos semestres/SEMESTRE 2/Aprendizaje automatico IV/Competencia-CNN/Kaggle_CNN_competition_ML4_MIAA/data
IMG_DIR: /Users/jhonattan.reales/Library/Mobile Documents/com~apple~CloudDocs/MAESTRIA MIAA 2025-2026/Cursos semestres/SEMESTRE 2/Aprendizaje automatico IV/Competencia-CNN/Kaggle_CNN_competition_ML4_MIAA/data/images
OUTPUT_DIR: /Users/jhonattan.reales/Library/Mobile Documents/com~apple~CloudDocs/MAESTRIA MIAA 2025-2026/Cursos semestres/SEMESTRE 2/Aprendi

In [4]:
# Read the CSV file
csv_file = os.path.join(DATA_DIR, "train.csv")
df_train = pd.read_csv(csv_file)

# Base CNN 
basic backbone + basic decision head

# Tunning the base model

## Improve the backbone architecture
Add more hidden layers and useful modules

## Improve the desicion head architecture
Add more layers or useful modules

## Hiperparameter tunning
Test the results by tunning different values of hiperparameters:
- d
- d

## Test Data augmentations
Use different data augmentation techniques and identify those who are better to improve the performance.

# Conclusions

----

In [ ]:
# # Ejemplo

# backbone = BaseBackbone()

# # Add an extra conv layer at the end of self.body
# extra_layer = nn.Sequential(
#     nn.Conv2d(4*32, 8*32, 3, 1, 1),
#     nn.BatchNorm2d(8*32),
#     nn.ReLU(inplace=True),
# )

# # Concatenate to the existing body
# backbone.body = nn.Sequential(
#     *(list(backbone.body.children()) + list(extra_layer.children()))
# )

In [ ]:
# # ejemplo

# backbone = BaseBackbone()

# # Replace projection with a deeper MLP
# backbone.proj = nn.Sequential(
#     backbone.proj,                      # original linear layer
#     nn.ReLU(inplace=True),
#     nn.Linear(backbone.proj.out_features, 256),  # new layer
# )